In [1]:
# !pip install --upgrade openai

In [2]:
import pandas as pd
import numpy as np
import random
import json
import redis

import openai

In [3]:
import os

with open('/Users/elainewei/Desktop/chatgpt-api-key') as f:
    openai.api_key = f.read()
    os.environ["OPENAI_API_KEY"] = f.read()

In [4]:
# caching
redis_client = redis.Redis(host = 'localhost', port = 6379, db = 0)

In [5]:
df = pd.read_csv('results/sample_bios_after109nd.csv')
df.head()

,id,bio
0,E000293,A Representative from Connecticut; born in Oak...
1,K000394,A Representative from New Jersey; born in Bost...
2,T000478,A Representative from New York; born in New Ha...
3,M000523,A Representative from Georgia; born in Atlanta...
4,F000479,"a Senator from Pennsylvania; born in Reading, ..."


In [6]:
sample_ids = list(df.id[3:6])
sample_bios = list(df.bio[3:6])

In [7]:
prompt_pre = open("prompt.txt", "r").read()

In [8]:
# # example
# completion = openai.ChatCompletion.create(
#     model='gpt-3.5-turbo', 
#     messages=[{'role':'user', 'content': prompt_pre + sample_bios[2]}]
# )
# print(completion)

In [9]:
# generalized function
def gen_table(id, bio, update=None):
    """
    Takes in a bio, let chatGPT generate the table, and outputs the table
    """
    table_key = f"politician_{id}"
    table = redis_client.get(table_key)

    if update:
        table = None

    if not table:
        print('Could not find table in cache. Retrieving from Chat GPT API...')
        prompt = prompt_pre + bio
        completion = openai.ChatCompletion.create(
            model='gpt-3.5-turbo', 
            temperature=0,
            messages=[{'role':'user', 'content': prompt}]
        )

        table = completion.choices[0].message.content
        redis_client.set(table_key, json.dumps(table))
    
    else:
        print('Found table in cache, serving from redis...')
        table = json.loads(table)

    return table

In [10]:
tables = []
for i in range(len(sample_bios)):
    table = gen_table(sample_ids[i], sample_bios[i], update=True)
    tables.append(table)

Could not find table in cache. Retrieving from Chat GPT API...
Could not find table in cache. Retrieving from Chat GPT API...
Could not find table in cache. Retrieving from Chat GPT API...


In [11]:
sample = ''
for i in range(len(tables)):
    sample += sample_ids[i] + '\n'
    sample += sample_bios[i] + '\n'
    sample += tables[i] + '\n'
print(sample)

M000523
A Representative from Georgia; born in Atlanta, Fulton County, Ga., March 17, 1955; graduated St. Joseph High School; B.A., University of Southern California, Los Angeles, Calif., 1978; attended Fletcher School of Law and Diplomacy, Medford, Mass.; diplomatic fellow, Spellman College, Atlanta, Ga., 1984; faculty member, Clark Atlanta University and Agnes Scott College; member of the Georgia state house of representatives, 1988-1992; elected as a Democrat to the One Hundred Third and to the four succeeding Congresses (January 3, 1993-January 3, 2003); unsuccessful candidate for nomination to the One Hundred Eighth Congress in 2002; elected as a Democrat to the One Hundred Ninth Congress (January 3, 2005-January 3, 2007); unsuccessful candidate for nomination to the One Hundred Tenth Congress in 2006; unsuccessful Green Party candidate for election for President of the United States in 2008.
Time Period | Location
--- | ---
1955-1971 | Atlanta, Fulton County, Ga.
1971-1978 | Los 

In [12]:
with open('results/sample_tables.txt', 'w') as f:
    f.write('sample')